# Run the Bayesian Online Change-Point Detector vs. the Benchmarks

In [6]:
# Load imports
import numpy as np 
import pandas as pd 
import itertools

from dev.Simulator import Simulator
from dev.stratagy import *
from dev.utils import NumericalTools

**Load Data**

In [7]:
returns = pd.read_csv("returns", sep=",", index_col="Date")
returns.tail()

,AAPL,ADBE,AMZN,BAC,CSCO,CVX,DIS,GOOGL,HD,INTC,...,BARC.L,RIO.L,TLT,IEF,SHY,BWX,PHAU.L,GLD,XLE,BTC-USD
Date,,,,,,,,,,,,,,,,,,,,,
2025-05-23,-0.030711,-0.016156,-0.010443,-0.002543,-0.003954,0.009197,-0.012769,-0.014145,-0.009385,-0.024632,...,-0.010295,-0.005342,0.001657,0.002884,0.000606,0.012717,0.019181,0.021670,0.003176,-0.040063
2025-05-27,0.024984,0.013183,0.024718,0.023337,0.010090,0.009403,0.023776,0.025956,0.020980,0.024632,...,0.016906,-0.011959,0.013860,0.004151,0.000364,-0.001744,-0.018692,-0.017094,0.008622,-0.004081
2025-05-28,0.001048,-0.002108,-0.006330,-0.003625,-0.006452,-0.013219,-0.007504,-0.003128,-0.006338,-0.008798,...,-0.016561,-0.014447,-0.004325,-0.002765,-0.000727,-0.003936,-0.000342,-0.002269,-0.012778,-0.010999
2025-05-29,-0.002348,0.002737,0.004776,0.004077,-0.004589,0.013799,0.004473,-0.002905,0.000625,-0.005908,...,0.000386,-0.003832,0.009212,0.004145,0.001091,0.006987,0.006364,0.005907,0.007443,-0.020246
2025-05-30,0.004491,0.004177,-0.003360,-0.002490,-0.000159,-0.008813,0.009064,-0.000698,0.000000,-0.035180,...,0.009452,-0.008300,0.001508,0.002965,0.001211,-0.001742,-0.008875,-0.006599,-0.008914,-0.015677


### Start with running all the stratagies

In [8]:
# Split the dataframe into train / validation split
df_fit = returns.loc[returns.index < '2008-01-01']
df_val = returns.loc[returns.index >= '2008-01-01']

df_fit.head()

,AAPL,ADBE,AMZN,BAC,CSCO,CVX,DIS,GOOGL,HD,INTC,...,BARC.L,RIO.L,TLT,IEF,SHY,BWX,PHAU.L,GLD,XLE,BTC-USD
Date,,,,,,,,,,,,,,,,,,,,,


**Run Grid Search to find the best set of hyperparameters**

In [ ]:
# Setup for grid search
strategy_grids = {
    "variable_window_with_variable_learning": (
        BOCPD.BOCPD,
        {
            "dim": [len(df_fit.columns)],
            "bocpd_params": [
                {
                    'p_c': p_c,
                    'theta_cp': theta_cp,
                    'max_run_length': max_run_length,
                    'mu0': mu0,
                    'kappa0': kappa0,
                    'psi0': psi0,
                    'nu0': nu0
                }
                for p_c in [0.0005, 0.001, 0.005, 0.01, 0.02]
                for theta_cp in [0.7, 0.8, 0.9, 0.95]
                for max_run_length in [200]
                for mu0 in [None]
                for kappa0 in [None]
                for psi0 in [None]
                for nu0 in [None]
            ],
            "opt_params": [
                {
                    'lam': lam,
                    'kappa': kappa,
                    'w_min': w_min,
                    'w_max': w_max
                }
                for lam in [0.1, 1, 5, 10, 20]
                for kappa in [0.1, 0.5, 0.8, 1, 1.5, 2, 5, 10]
                for w_min in [0.05, 0.1, 0.14]
                for w_max in [0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]
            ],
            "mu0": [None],
            "Sigma0": [None]
        }
    ),
    "recursive_EWMA": (
        BFS.BFSOnlyStrategy,
        {
            "span":        [10, 20, 50],
            "min_periods": [1, 5, 10],
        }
    ),
    "volatility_window": (
        VAW.VolatilityAdaptiveWindowStrategy,
        {
            "initial_window": [30, 60, 90],
            "kappa":          [1.0, 1.5, 2.0],
        }
    ),
    "fixed_sliding_window": (
        FSW.FixedSlidingWindowStrategy,
        {
            "window_size": [50, 100, 200],
        }
    ),
    "oracle_regime_model": (
        ORACLE.OracleRegimeStrategy,
        {
            "lookahead": [1, 2, 5],
        }
    ),
    "equal_weight_portfolio": (
        EqualWeight.EqualWeightStrategy,
        {
            # No hyperparameters for equal-weight; include dummy key
            "dummy": [None],
        }
    ),
}

"""
Start & Execute Grid-Search
"""
best_overall = {}

for name, (StratClass, param_grid) in strategy_grids.items():
    keys = list(param_grid.keys())
    value_lists = [param_grid[k] for k in keys]
    
    best_sharpe = -np.inf
    best_combo = None

    for combo in itertools.product(*value_lists):
        params = dict(zip(keys, combo))
        if "dummy" in params:
            params.pop("dummy")
        
        strat_obj = StratClass(**params) if params else StratClass()
        sim = Simulator(df_fit, strategy=strat_obj)
        sim.run()

        sharpe = NumericalTools.compute_sharpe(sim.returns_df)
        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_combo = params.copy()

    best_overall[name] = {
        "best_params": best_combo or {},
        "best_sharpe": best_sharpe
    }

# Select the "Best" hyperparams (those that maximised the annualised sharpe)
strategies = {}
for name, (StratClass, _) in strategy_grids.items():
    bp = best_overall[name]["best_params"]
    strategies[name] = (StratClass, bp)

KeyError: 'lam'

In [ ]:
simulators = {
    name: Simulator(df_val, strategy=cls(**params))
    for name, (cls, params) in strategies.items()
}

for sim in simulators.values():
    sim.run()

In [ ]:
best_table = pd.DataFrame.from_dict(
    {
        name: {**info["best_params"], "Sharpe(2005-2007)": info["best_sharpe"]}
        for name, info in best_overall.items()
    },
    orient="index"
)